# [STARTER] Udaplay Project

## Part 01 - Offline RAG

In this part of the project, you'll build your VectorDB using Chroma.

The data is inside folder `project/starter/games`. Each file will become a document in the collection you'll create.
Example.:
```json
{
  "Name": "Gran Turismo",
  "Platform": "PlayStation 1",
  "Genre": "Racing",
  "Publisher": "Sony Computer Entertainment",
  "Description": "A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.",
  "YearOfRelease": 1997
}
```


### Setup

In [4]:
pip install "chromadb>=1.0.4" "openai>=1.73.0" "pydantic>=2.11.3" "python-dotenv>=1.1.0" "tavily-python>=0.5.4"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv

In [12]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("BASE_URL")

### VectorDB Instance

In [13]:
# Instantiate your ChromaDB Client
chroma_client = chromadb.PersistentClient(path="chromadb")

### Collection

In [16]:
# Pick one embedding function
embedding_fn = embedding_functions.OpenAIEmbeddingFunction()

In [17]:
# Create a collection
collection = chroma_client.create_collection(
   name="udaplay",
   embedding_function=embedding_fn
)



### Add documents

In [18]:
# Make sure you have a directory "project/starter/games"
data_dir = "games"

for file_name in sorted(os.listdir(data_dir)):
    if not file_name.endswith(".json"):
        continue

    file_path = os.path.join(data_dir, file_name)
    with open(file_path, "r", encoding="utf-8") as f:
        game = json.load(f)

    # You can change what text you want to index
    content = f"[{game['Platform']}] {game['Name']} ({game['YearOfRelease']}) - {game['Description']}"

    # Use file name (like 001) as ID
    doc_id = os.path.splitext(file_name)[0]

    collection.add(
        ids=[doc_id],
        documents=[content],
        metadatas=[game]
    )

In [20]:
#Implement semantic search functionality
findByName = collection.query(
    query_texts=["pokemon"],
    n_results=3
)
print(findByName)

# Search by genere
findByGenre = collection.query(
    query_texts=["nintendo"],
    n_results=3,
    where={"Genre": "Fighting"}
)
print(findByGenre)


{'ids': [['006', '007', '009']], 'embeddings': None, 'documents': [['[Game Boy Color] Pokémon Gold and Silver (1999) - Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.', '[Game Boy Advance] Pokémon Ruby and Sapphire (2002) - Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.', "[Nintendo 64] Super Mario 64 (1996) - A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'Publisher': 'Nintendo', 'YearOfRelease': 1999, 'Name': 'Pokémon Gold and Silver', 'Platform': 'Game Boy Color', 'Genre': 'Role-playing', 'Description': 'Second-generation Pokémon games introducing new regions, Pokémon, and gameplay mechanics.'}, {'Description': 'Third-generation Pokémon games set in the Hoenn region, featuring new Pokémon and double battles.', 'Platform':

In [28]:
findByPlatform = collection.get(
      limit=10,
      offset=0,
      where={"Platform": "Nintendo Switch"}
  )

print(findByPlatform)

{'ids': ['012'], 'embeddings': None, 'documents': ['[Nintendo Switch] Mario Kart 8 Deluxe (2017) - An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [{'Platform': 'Nintendo Switch', 'Publisher': 'Nintendo', 'Genre': 'Racing', 'Name': 'Mario Kart 8 Deluxe', 'Description': 'An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics.', 'YearOfRelease': 2017}]}
